In [12]:
import os
CURRENT_FOLDER = os.getcwd()
WORKDIR = os.path.dirname(CURRENT_FOLDER)
#os.environ['PYTHONPATH'] = WORKDIR
os.environ['PYTHONPATH'] = '/home/clarkmaio/workspace/AuroraBorealis

print(os.getcwd())

/home


In [11]:
import pandas as pd
import os
from src import DataScraper
import holoviews as hv
import seaborn as sns
import matplotlib.pyplot as plt
hv.extension('bokeh')

ModuleNotFoundError: No module named 'src'

In [ ]:
df = DataScraper().load_data()

-------------------  DATA SOURCE -------------------
# LICENSE: CC BY 4.0
# SOURCE: Geomagnetic Observatory Niemegk, GFZ German Research Centre for Geosciences
# PLEASE CITE: Matzka, J., Stolle, C., Yamazaki, Y., Bronkalla, O. and Morschhauser, A., 2021. The geomagnetic Kp index 
# and derived indices of geomagnetic activity. Space Weather, https://doi.org/10.1029/2020SW002641
----------------------------------------------------


In [ ]:
df.head()

,Kp,ap
valuedate,,
1932-01-01 00:00:00,3.333000,18.000000
1932-01-01 01:00:00,3.093164,15.835427
1932-01-01 02:00:00,2.867353,13.801675
1932-01-01 03:00:00,2.667000,12.000000
1932-01-01 04:00:00,2.503536,10.531658


In [ ]:
df_month = df.resample('MS').mean()
df_month['month'] = df_month.index.month
df_month['year'] = df_month.index.year

df_pivot = df_month.pivot_table(index='month', columns='year', values='Kp')

In [ ]:
# Create a heatmap
# Make square heatmap cell
fig = plt.figure(figsize=(12, 6))
sns.heatmap(df_pivot, cmap='coolwarm', linewidths=0.5, square=True, cbar_kws={'label': 'Kp'})

plt.title('Monthly average Kp', fontweight='bold')
plt.xlabel('Year', fontweight='bold')
plt.ylabel('Month', fontweight='bold')

NameError: name 'plt' is not defined

In [ ]:
# Plot Kp with holoviews
plot = hv.Area(df.reset_index().iloc[-10000:, :], 'valuedate', 'Kp').opts(color='red', line_color='black', tools=['hover'], alpha=0.5)
plot.opts(width=1500, height=600, title='Kp index')
plot


:Area   [valuedate]   (Kp)